#### Reference: https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html#learning-methods-on-graphs

In [ ]:
from torch_geometric.data import Data
import json
from collections import Counter
import torch
from tqdm import tqdm
from torch_geometric.nn import SGConv

In [ ]:
def read_json_to_list(pth):
    res = []
    with open(pth, "r") as fin:
        for line in fin:
            res.append(json.loads(line.strip()))
    return res       

In [ ]:
hrgs = read_json_to_list("/usr0/home/amadaan/data/audio/LJSpeech-1.1/TTS/hrg.jsonl")

In [ ]:
len(hrgs)

In [ ]:
hrgs[0]

## Make vocab, init random embeddings

In [ ]:
def get_tokens(hrgs):
    tokens = []
    for hrg in hrgs:
        for word_rep in hrg["hrg"]:
            tokens.extend(get_tokens_from_word_rep(word_rep))
    return tokens

def get_tokens_from_word_rep(word_rep):
    tokens = []
    tokens.append(word_rep["word"])
    for daughter in word_rep["daughters"]:
        tokens.append(daughter["syll"])
    return tokens

In [ ]:
def make_vocab(hrgs):
    tokens = Counter(list(get_tokens(hrgs)))
    
    tokens = [w[0] for w in tokens.items() if w[1] > 1]
    tokens.extend([str(i) for i in range(20)])  # position
    tokens.extend(["<W>", "<SYLL>", "<UNK>"])
    tok2id = {w:i for i, w in enumerate(tokens)}
    id2tok = {i:w for w, i in tok2id.items()}
    
    return tok2id, id2tok
    

In [ ]:
tok2id, id2tok = make_vocab(hrgs)

In [ ]:
def get_tok2id(tok):
    if tok in tok2id:
        return tok2id[tok]
    return tok2id["<UNK>"]

In [ ]:
n_embed = 64

In [ ]:
hrgs[0]

In [ ]:
embeddings = torch.rand(len(tok2id), n_embed)

### Convert HRGs to PyTorchGeom Objects

In [ ]:
def hrg_to_graph(hrg):
    """
    Converts the HRG to graph,
    
    Returns:
        Edge index: (num_edges, 2)
        Node features: (num_nodes, feature_dim)
    """
    words, sylls = [], []
    node_idx = {}
    edges = []
    node_features = []
    syll_node_idxs = set()
    for i, word_rep in enumerate(hrg["hrg"]):
        word_node = f"{word_rep['word']}-{i}"
        word_node_id = get_tok2id(word_rep['word'])
        
        node_idx[word_node] = len(node_idx)
        node_features.append(embeddings[word_node_id, :])
        for j, syll in enumerate(word_rep["daughters"]):
            syll_node = f"{syll['syll']}-{i}-{j}"
            syll_node_id = get_tok2id(syll['syll'])
            
            node_idx[syll_node] = len(node_idx)
            syll_node_idxs.add(node_idx[syll_node])
            
            node_features.append(embeddings[syll_node_id, :])
            edges.append([node_idx[word_node], node_idx[syll_node]])
    return torch.tensor(edges, dtype=torch.long), torch.stack(node_features).float(),\
           torch.tensor(list(syll_node_idxs), dtype=torch.long)

In [ ]:
hrg_to_graph(hrgs[0])[2]

In [ ]:
py_geom_graphs = []
for hrg in tqdm(hrgs, total=len(hrgs)):
    edge_index, node_features, syll_nodes = hrg_to_graph(hrg)
    data = Data(x=node_features, edge_index=edge_index.t().contiguous(), syll_nodes=syll_nodes)
    py_geom_graphs.append(data)
    

In [ ]:
from torch_geometric.data import DataLoader

In [ ]:
loader = DataLoader(py_geom_graphs, batch_size=32, shuffle=True)


In [ ]:
batches = []
for batch in loader:
    batches.append(batch)

In [ ]:
type(batches[0])

In [ ]:
from torch_geometric.data.batch import Batch

In [ ]:
Batch.from_data_list(batches[0].to_data_list())

In [ ]:
batches[0]

In [ ]:
x = 0
for g in batches[0].to_data_list():
    x += g.x.shape[0]
print(x)

### Sample GCN

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(n_embed, 16)
        self.conv2 = GCNConv(16, 200)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [ ]:
conv1 = GCNConv(n_embed, 16).cuda()

In [ ]:
x, edge_index = batches[0].x, batches[0].edge_index

In [ ]:
conv1 = SGConv(
            n_embed, 16, K=5, cached=True).cuda()

In [ ]:
x = conv1(x.cuda(), edge_index.cuda())

In [ ]:
x

In [ ]:
type(batch.x)

In [ ]:
edge_index.shape

In [ ]:
x.shape

In [ ]:
x = conv1(x.cuda(), edge_index.cuda())

In [ ]:
x.shape

In [ ]:
batches[0].to_data_list()[0].syll_nodes.shape

In [ ]:
batches[0].to_data_list()

In [ ]:
x.shape

In [ ]:
batch_graphs = batches[0].to_data_list()

In [ ]:
batch_graphs[0].x.shape[0]

In [ ]:
batch_graphs[0].syll_nodes.shape

In [ ]:
offset = 0

In [ ]:
x[offset + batch_graphs[0].syll_nodes]

In [ ]:
offset += batch_graphs[0].x.shape[0]

In [ ]:
x[offset + batch_graphs[1].syll_nodes].shape

In [ ]:
??torch_geometric.data.batch.Batch

In [ ]:
def break_into_utterances(x, batch):
    offset = 0
    res = []
    batch_graphs = batch.to_data_list()
    for graph in batch_graphs:
        res.append(x[offset + graph.syll_nodes])
        offset += graph.x.shape[0]
    return res

In [ ]:
res = break_into_utterances(x, batches[0])

In [ ]:
res